In [1]:
from bayes_opt import BayesianOptimization
import pandas as pd
import numpy as np
import h5py
import matplotlib.pyplot as plt 
# from scipy import stats
# from scipy.special import boxcox, inv_boxcox
from tqdm import tqdm 
# import sklearn.datasets
import sklearn.model_selection
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from multiprocessing import Pool 


In [2]:
with h5py.File('MOS_Scores.mat', 'r') as f:
    for k,v in f.items():
        Y = (np.array(v))
        Y = Y.reshape(Y.shape[1])
        np.save('Y_mat.npy',Y)
with h5py.File('konvid_features.mat', 'r') as f:
    for k,v in f.items():
        X = (np.array(v)).T
        np.save('X_mat.npy',X)

        
        X = StandardScaler().fit_transform(X)
        imp_cols = np.array([2,9,17,23,27,30,41,47,50,51,52,54,57,59,65,67,68,70,72,74,75,78,83,85,90,91,99,100,101])-2
#         X = X[:,imp_cols]
        
        print(X.shape)
        
x_train,x_test,y_train,y_test = sklearn.model_selection.train_test_split(X,Y,test_size=0.2,shuffle=True,random_state=42)
x_train,x_val,y_train,y_val = x_train[:900].copy(), x_train[900:].copy(),y_train[:900].copy(),y_train[900:].copy()

print(x_train.shape,x_val.shape,x_test.shape)
print(y_train.shape,y_val.shape,y_test.shape)

(1200, 100)
(900, 100) (60, 100) (240, 100)
(900,) (60,) (240,)


In [3]:
imp_cols = np.array([2,9,17,23,27,30,41,47,50,51,52,54,57,59,65,67,68,70,72,74,75,78,83,85,90,91,99,100,101])-2
rem_cols = np.setdiff1d(np.arange(100),imp_cols)

Simple Neural Net Based Approach

In [4]:
# from keras.models import Sequential
# from keras.layers import Dense
# from keras import regularizers


In [5]:
# model = Sequential()
# model.add(Dense(25, input_dim=100, kernel_regularizer=regularizers.l2(0.01),activation='elu'))
# model.add(Dense(20, kernel_regularizer=regularizers.l2(0.01),activation='elu'))
# model.add(Dense(15, kernel_regularizer=regularizers.l2(0.01),activation='elu'))
# model.add(Dense(10,kernel_regularizer=regularizers.l2(0.01),activation='elu'))
# model.add(Dense(5,activation='elu'))
# model.add(Dense(1, activation='elu'))

In [6]:
# model.compile(loss='mse', optimizer='adam', metrics=['mse'])

In [7]:
# model.fit(x_train, y_train, epochs=50, validation_data=(x_val,y_val),batch_size=4)

In [8]:
# y_pred = model.predict(x_test)
# y_pred = y_pred.reshape(y_pred.shape[0])
# print(y_pred.shape,y_test.shape)
# print(np.corrcoef(y_test,y_pred)[0,1])

In [9]:
# imp_cols = np.array([2,9,17,23,27,30,41,47,50,51,52,54,57,59,65,67,68,70,72,74,75,78,83,85,90,91,99,100,101])-2
# X1 = np.hstack((X[:,imp_cols],X[:,imp_cols]**2,X[:,imp_cols]**0.5))

In [10]:
# print(y_pred[:10])
# print(y_test[:10])

In [11]:
# plt.plot(y_pred[:10])
# plt.plot(y_test[:10])

In [12]:
# plt.hist(y_pred-y_test,bins=50)

Gaussian Process Regression

In [13]:
import math
import torch
import gpytorch

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [14]:
train_x = torch.tensor(x_train.astype(np.float32))
train_y = torch.tensor(y_train.astype(np.float32))
test_x = torch.tensor(np.concatenate((x_val,x_test)).astype(np.float32))
test_y = torch.tensor(np.concatenate((y_val,y_test)).astype(np.float32))
print(train_x.shape,train_y.shape,test_x.shape,test_y.shape)

torch.Size([900, 100]) torch.Size([900]) torch.Size([300, 100]) torch.Size([300])


In [28]:
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ZeroMean()
#         self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel(ard_num_dimns=x_train.shape[1]))
#         self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.MaternKernel(nu=2.5,ard_num_dimns=x_train.shape[1]) + gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel()))
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.MaternKernel(nu=2.5,ard_num_dims=len(imp_cols),active_dims=imp_cols) * gpytorch.kernels.MaternKernel(active_dims=rem_cols))

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

# initialize likelihood and model
likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = ExactGPModel(train_x, train_y, likelihood)

In [40]:
# Find optimal model hyperparameters
model.train()
likelihood.train()

# Use the adam optimizer
optimizer = torch.optim.Adam([{'params': model.parameters()},],lr=0.001)

# "Loss" for GPs - the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

training_iters = 50
for i in range(training_iters):
    # Zero gradients from previous iteration
    optimizer.zero_grad()
    # Output from model
    output = model(train_x)
    # Calc loss and backprop gradients
    loss = -mll(output, train_y)
    loss.backward()
    print('Iter %d/%d - Loss: %.3f' % (
            i + 1, training_iters, loss.item()))
    optimizer.step()

Iter 1/50 - Loss: 0.707
Iter 2/50 - Loss: 0.701
Iter 3/50 - Loss: 0.704
Iter 4/50 - Loss: 0.699
Iter 5/50 - Loss: 0.705
Iter 6/50 - Loss: 0.704
Iter 7/50 - Loss: 0.700
Iter 8/50 - Loss: 0.703
Iter 9/50 - Loss: 0.700
Iter 10/50 - Loss: 0.714
Iter 11/50 - Loss: 0.706
Iter 12/50 - Loss: 0.705
Iter 13/50 - Loss: 0.703
Iter 14/50 - Loss: 0.704
Iter 15/50 - Loss: 0.712
Iter 16/50 - Loss: 0.709
Iter 17/50 - Loss: 0.705
Iter 18/50 - Loss: 0.700
Iter 19/50 - Loss: 0.709
Iter 20/50 - Loss: 0.704
Iter 21/50 - Loss: 0.700
Iter 22/50 - Loss: 0.703
Iter 23/50 - Loss: 0.710
Iter 24/50 - Loss: 0.699
Iter 25/50 - Loss: 0.698
Iter 26/50 - Loss: 0.703
Iter 27/50 - Loss: 0.708
Iter 28/50 - Loss: 0.711
Iter 29/50 - Loss: 0.705
Iter 30/50 - Loss: 0.700
Iter 31/50 - Loss: 0.710
Iter 32/50 - Loss: 0.701
Iter 33/50 - Loss: 0.706
Iter 34/50 - Loss: 0.708
Iter 35/50 - Loss: 0.701
Iter 36/50 - Loss: 0.713
Iter 37/50 - Loss: 0.705
Iter 38/50 - Loss: 0.701
Iter 39/50 - Loss: 0.708
Iter 40/50 - Loss: 0.716
Iter 41/5

In [41]:
for param_name, param in model.named_parameters():
    print(f'Parameter name: {param_name:42} value = {param.item()}')

Parameter name: likelihood.noise_covar.raw_noise           value = -1.6555365324020386
Parameter name: covar_module.raw_outputscale               value = 1.9082797765731812


ValueError: only one element tensors can be converted to Python scalars

In [42]:
model.eval()
likelihood.eval()

f_preds = model(test_x)
y_pred = f_preds.mean

print(y_pred.shape,y_test.shape)
print("pearson",np.corrcoef(test_y.detach().numpy(),y_pred.detach().numpy())[0,1])

torch.Size([300]) (240,)
pearson 0.6894163204202093


In [43]:
a=test_y.detach().numpy()
b=y_pred.detach().numpy()
import scipy.stats
print("Spearman",scipy.stats.spearmanr(a, b, axis=0)[0])

Spearman 0.6897298990832388
